In [5]:
# Extract 3-hourly 500 hPa Relative Humidity from regridded vertical velocity (hur) data of UM zoom 6 model
# Extract 6-hourly 500 hPa Relative Humidity from regridded vertical velocity (hur) data of ICON zoom 6 model
import xarray as xr
from pathlib import Path
import easygems.healpix as egh
import healpy as hp
import numpy as np

p = 500 # hPa; level of interest
data = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT3H.z6.zarr" #PT3H is 3-hourly data
data = "/g/data/qx55/germany_node/d3hp003.zarr/PT6H_point_z6_atm.zarr" #PT6H is 6-hourly data
ds = xr.open_zarr(data)

#r = ds['hur'].sel(pressure=p)
#r = ds['hur'].sel(pressure=p)* 100 # to %
r = ds['hur'].sel(pressure=p*100)  # to %

def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[("latitude", lats), ("longitude", lons)],
    )

In [6]:
this_nside = hp.get_nside(r)
res = 1  # resolution
#Darwin lat lon: -12.4, 130.85
lon = np.arange(130.5, 131.5 + res, res)
lat = np.arange(-13, -12 + res, res)

cells = get_nn_lon_lat_index(this_nside, lon, lat) 
r_regrid = r.isel(cell=cells)
r_regrid

<xarray.DataArray 'hur' (time: 1700, latitude: 2, longitude: 2)> Size: 27kB
[6800 values with dtype=float32]
Coordinates:
    pressure   int64 8B 50000
  * time       (time) datetime64[ns] 14kB 2020-01-01T06:00:00 ... 2021-03-01
  * latitude   (latitude) int64 16B -13 -12
  * longitude  (longitude) float64 16B 130.5 131.5
Attributes:
    grid_mapping:        crs
    hiopy::time_method:  point
    hiopy::src_name:     pl::hur
    hiopy::nnn:          4
    hiopy::enable:       True
    standard_name:       relative_humidity
    units:               
    short_name:          
    long_name:           relative humidity

In [7]:
import pandas as pd
# Create full date range from 2020-03-01 to 2021-02-28
#full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="3h")
full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="6h")

# Define exclusion period
exclude_start = pd.Timestamp("2020-05-01 00:00:00")
exclude_end = pd.Timestamp("2020-09-30 00:00:00")

# Filter out the excluded range
filtered_range = full_range[(full_range < exclude_start) | (full_range > exclude_end)]

# Convert to list of strings in yyyymmdd format
date_str_list = filtered_range.strftime("%Y%m%d %H:%M:%S").tolist()

r500 = r_regrid.sel(time=date_str_list).mean(dim=("latitude","longitude"))
r500

<xarray.DataArray 'hur' (time: 848)> Size: 3kB
array([77.38853  , 51.893517 , 59.562702 , 67.79706  , 70.57184  ,
       68.11294  , 66.68614  , 84.093636 , 88.447586 , 90.65008  ,
       91.7756   , 91.70941  , 84.47965  , 72.91698  , 61.866302 ,
       54.663414 , 39.587105 , 33.333923 , 36.704067 , 55.922607 ,
       60.253483 , 67.686325 , 70.98242  , 87.09394  , 88.25719  ,
       80.037285 , 85.825745 , 90.41457  , 80.872055 , 78.78773  ,
       85.94819  , 90.73675  , 83.10341  , 76.75523  , 79.906494 ,
       79.93942  , 70.684044 , 65.55555  , 67.1867   , 68.66173  ,
       58.06003  , 61.993507 , 57.281124 , 59.520386 , 60.756664 ,
       71.31926  , 73.51262  , 88.358246 , 87.19674  , 83.40104  ,
       87.560844 , 92.96086  , 88.95174  , 83.57621  , 88.24783  ,
       86.06058  , 79.05565  , 80.90147  , 89.27334  , 91.60593  ,
       85.955055 , 78.2781   , 86.95874  , 93.121506 , 85.90808  ,
       77.8459   , 81.45433  , 72.33453  , 65.52362  , 47.887688 ,
       39.71332  , 35.703075 , 25.414196 , 15.904671 ,  8.402922 ,
        5.8190556,  6.886589 ,  8.640583 , 15.081333 , 22.324825 ,
       28.10896  , 38.99684  , 66.77795  , 74.21939  , 61.384567 ,
       71.499794 , 82.489235 , 88.3614   , 76.99402  , 72.5172   ,
       79.6445   , 86.132454 , 75.42844  , 69.760956 , 63.20169  ,
       74.48573  , 84.065834 , 75.75397  , 74.18547  , 87.49184  ,
...
       95.78075  , 92.481766 , 88.170204 , 96.16441  , 95.24286  ,
       91.63999  , 90.156944 , 89.87993  , 92.32282  , 93.51736  ,
       90.79149  , 94.719795 , 94.26578  , 95.29305  , 91.36853  ,
       92.83427  , 96.52778  , 95.83839  , 94.84336  , 95.56186  ,
       93.00897  , 96.560425 , 96.582504 , 96.71625  , 96.699524 ,
       96.58882  , 96.992    , 96.9424   , 95.773834 , 96.50085  ,
       96.22014  , 96.45098  , 95.5741   , 95.551346 , 90.62473  ,
       95.33893  , 96.510185 , 94.25008  , 92.08995  , 90.46249  ,
       90.82817  , 92.46672  , 94.37128  , 95.934494 , 96.384125 ,
       92.159195 , 89.158226 , 86.05626  , 80.8303   , 66.656654 ,
       62.88363  , 67.077156 , 69.56265  , 64.449135 , 65.613594 ,
       68.864624 , 71.17077  , 76.199356 , 78.50561  , 84.13468  ,
       91.99628  , 83.03945  , 87.18644  , 90.979294 , 92.41244  ,
       85.051544 , 84.90284  , 87.10133  , 86.81145  , 91.19847  ,
       89.17997  , 84.50839  , 86.546295 , 85.77865  , 80.768524 ,
       74.969345 , 80.846695 , 79.13983  , 68.4533   , 64.60281  ,
       80.65323  , 79.583984 , 71.54809  , 73.36462  , 76.42549  ,
       86.92404  , 84.99632  , 45.49691  , 24.03769  , 41.379475 ,
       43.12806  , 53.539196 , 66.881485 , 82.601814 , 84.96485  ,
       87.786446 , 83.346146 , 79.64978  ], dtype=float32)
Coordinates:
    pressure  int64 8B 50000
  * time      (time) datetime64[ns] 7kB 2020-03-01 ... 2021-02-28

In [4]:
dir = '/scratch/nf33/hk25_LSP/UK/Darwin/'
dir = '/scratch/nf33/hk25_LSP/Germany/Darwin/'
fname = 'RH500_3hr.nc'
fname = 'RH500_6hr.nc'

r500.to_netcdf(dir+fname)
print(f"Saved to ",dir+fname)

Saved to  /scratch/nf33/hk25_LSP/Germany/Darwin/RH500_6hr.nc
